# Detectando objetos em vídeos com YOLOv4 e OpenCV 

## Etapa 1 - Importando as bibliotecas

In [1]:
import cv2
print(cv2.__version__)

4.6.0


In [2]:
!pip install opencv-python==4.4.0.40

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66


In [1]:
import cv2
import numpy as np
import time
import os
import matplotlib.pyplot as plt 
from google.colab.patches import cv2_imshow
import zipfile
print(cv2.__version__)

4.4.0


## Etapa 2 - Conectando com o Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Etapa 3 - Carregando os arquivos do modelo treinado 

In [3]:
path =  '/content/gdrive/My Drive/YOLO/modelo_YOLOv4.zip'
zip_object = zipfile.ZipFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close()

In [4]:
labels_path = os.path.sep.join(['/content/cfg', "coco.names"])
LABELS = open(labels_path).read().strip().split("\n")

In [5]:
weights_path = os.path.sep.join(['/content/', "yolov4.weights"])
config_path = os.path.sep.join(['/content/cfg', "yolov4.cfg"])

In [6]:
net = cv2.dnn.readNet(config_path, weights_path)

## Etapa 4 - Definindo mais configurações para a detecção 

In [7]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

In [8]:
ln = net.getLayerNames()
print("Todas as camadas (layers):")
print(ln)
print("Total: "+ str(len(ln)))
print("Camadas de saída: ")
print(net.getUnconnectedOutLayers())
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

Todas as camadas (layers):
['conv_0', 'bn_0', 'mish_1', 'conv_1', 'bn_1', 'mish_2', 'conv_2', 'bn_2', 'mish_3', 'identity_3', 'conv_4', 'bn_4', 'mish_5', 'conv_5', 'bn_5', 'mish_6', 'conv_6', 'bn_6', 'mish_7', 'shortcut_7', 'conv_8', 'bn_8', 'mish_9', 'concat_9', 'conv_10', 'bn_10', 'mish_11', 'conv_11', 'bn_11', 'mish_12', 'conv_12', 'bn_12', 'mish_13', 'identity_13', 'conv_14', 'bn_14', 'mish_15', 'conv_15', 'bn_15', 'mish_16', 'conv_16', 'bn_16', 'mish_17', 'shortcut_17', 'conv_18', 'bn_18', 'mish_19', 'conv_19', 'bn_19', 'mish_20', 'shortcut_20', 'conv_21', 'bn_21', 'mish_22', 'concat_22', 'conv_23', 'bn_23', 'mish_24', 'conv_24', 'bn_24', 'mish_25', 'conv_25', 'bn_25', 'mish_26', 'identity_26', 'conv_27', 'bn_27', 'mish_28', 'conv_28', 'bn_28', 'mish_29', 'conv_29', 'bn_29', 'mish_30', 'shortcut_30', 'conv_31', 'bn_31', 'mish_32', 'conv_32', 'bn_32', 'mish_33', 'shortcut_33', 'conv_34', 'bn_34', 'mish_35', 'conv_35', 'bn_35', 'mish_36', 'shortcut_36', 'conv_37', 'bn_37', 'mish_38'

## Etapa 5 - Criando as funções para detecção e processamento do video 


### Função para exibir imagens no Colab

In [9]:
def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(16, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

### Construindo o blob da imagem


In [10]:
def blob_imagem(net, imagem, mostrar_texto=True):
  inicio = time.time() 
  blob = cv2.dnn.blobFromImage(imagem, 1 / 255.0, (416, 416), swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)
  termino = time.time()
  if mostrar_texto:
    print("YOLO levou {:.2f} segundos".format(termino - inicio))
  return net, imagem, layerOutputs

### Realizando a detecção

In [11]:
def deteccoes(detection, _threshold, caixas, confiancas, IDclasses):
  scores = detection[5:] 
  classeID = np.argmax(scores)  
  confianca = scores[classeID]

  if confianca > _threshold:
      caixa = detection[0:4] * np.array([W, H, W, H])     
      (centerX, centerY, width, height) = caixa.astype("int")
            
      x = int(centerX - (width / 2))
      y = int(centerY - (height / 2))

      caixas.append([x, y, int(width), int(height)])
      confiancas.append(float(confianca))
      IDclasses.append(classeID)
      
  return caixas, confiancas, IDclasses

### Mostrando o resultado da detecção no video 

In [12]:
def funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=True):  
  (x, y) = (caixas[i][0], caixas[i][1])
  (w, h) = (caixas[i][2], caixas[i][3])

  cor = [int(c) for c in COLORS[IDclasses[i]]]
  cv2.rectangle(imagem, (x, y), (x + w, y + h), cor, 2) 
  texto = "{}: {:.4f}".format(LABELS[IDclasses[i]], confiancas[i])
  if mostrar_texto:
    print("> " + texto)
    print(x,y,w,h)
  cv2.putText(imagem, texto, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor, 2)

  return imagem,x,y,w,h

## Etapa 6 - Carregando o vídeo onde será feita a detecção

### 6.1 - De uma url

In [13]:
!wget https://github.com/gabevr/yolo/raw/master/videos/video_pessoas01.mp4

--2023-01-28 00:56:47--  https://github.com/gabevr/yolo/raw/master/videos/video_pessoas01.mp4
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_pessoas01.mp4 [following]
--2023-01-28 00:56:47--  https://raw.githubusercontent.com/gabevr/yolo/master/videos/video_pessoas01.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6129513 (5.8M) [application/octet-stream]
Saving to: ‘video_pessoas01.mp4’

video_pessoas01.mp4 100%[===================>]   5.84M  --.-KB/s    in 0.02s   

2023-01-28 00:56:47 (276 MB/s) - ‘video_pessoas01.mp4’ saved [6129513/6129513]



### 6.2 - Do Google Drive

In [ ]:
!cp /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/videos/video_pessoas01.mp4 ./

### Lendo o arquivo de vídeo com o OpenCV

In [14]:
arquivo_video = 'video_pessoas01.mp4'
cap = cv2.VideoCapture(arquivo_video)
conectado, video = cap.read()

In [15]:
conectado

True

In [16]:
video.shape

(720, 1280, 3)

In [17]:
video_largura = video.shape[1]
video_altura = video.shape[0]
video_largura, video_altura

(1280, 720)

## Etapa 7 - Redimensionamento do tamanho do video (opcional)

In [18]:
def redimensionar(largura, altura, largura_maxima = 600): 
  if (largura > largura_maxima):
    proporcao = largura / altura
    video_largura = largura_maxima
    video_altura = int(video_largura / proporcao)
  else:
    video_largura = largura
    video_altura = altura

  return video_largura, video_altura

In [19]:
video_largura, video_altura = redimensionar(video.shape[1], video.shape[0])
print(video_largura,video_altura)

600 337


## Etapa 8 - Definindo as configurações do vídeo

- Mais exemplos de outras configurações com o fourcc que é possível usar: https://www.programcreek.com/python/example/89348/cv2.VideoWriter_fourcc

In [20]:
nome_arquivo = 'resultado.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID') # MP4V

In [21]:
fps = 24

In [22]:
saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))

## Etapa 9 - Definindo as variáveis

In [23]:
threshold = 0.5
threshold_NMS = 0.3
fonte_pequena, fonte_media = 0.4, 0.6
fonte = cv2.FONT_HERSHEY_SIMPLEX

In [24]:
amostras_exibir = 20
amostra_atual = 0

## Etapa 10 - Processamento do vídeo e exibição do resultado

In [25]:
while (cv2.waitKey(1) < 0):
  conectado, frame = cap.read()
  if not conectado:
    break
  t = time.time()
  frame = cv2.resize(frame, (video_largura, video_altura))
  try:
    (H, W) = frame.shape[:2]
  except:
    print('Erro')
    continue

  imagem_cp = frame.copy() 
  net, frame, layerOutputs = blob_imagem(net, frame)
  caixas = []       
  confiancas = []   
  IDclasses = []    

  for output in layerOutputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, threshold, caixas, confiancas, IDclasses)

  objs = cv2.dnn.NMSBoxes(caixas, confiancas, threshold, threshold_NMS)

  if len(objs) > 0:
    for i in objs.flatten():
      frame, x, y, w, h = funcoes_imagem(frame, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=False)
      objeto = imagem_cp[y:y + h, x:x + w]
  
  cv2.putText(frame, " frame processado em {:.2f} segundos".format(time.time() - t), 
              (20, video_altura-20), fonte, fonte_pequena, (250, 250, 250), 0, lineType=cv2.LINE_AA)
  
  if amostra_atual <= amostras_exibir:
    cv2_imshow(frame)
    amostra_atual += 1

  saida_video.write(frame)

print('Terminou')
saida_video.release()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

In [26]:
!du -h resultado.avi

13M	resultado.avi


In [27]:
!cp ./resultado.avi /content/gdrive/My\ Drive/YOLO/resultado4.avi